Import necessary libraries

In [1]:
import numpy as py
import pandas as pd

Load and Prep Data

In [3]:
#Load election results
election_results = pd.read_csv('Results.csv')
#Load new constituencies
file = 'New Constituencies.xlsx'
xl = pd.ExcelFile(file)
new_constituencies = xl.parse('Sheet9')

In [4]:
#Separate the election results into preliminary and validated 
#For this project, we will only be using validated election results
results_val = election_results.loc[election_results['Type of results'] == 'validated']

results_prelim = election_results.loc[election_results['Type of results'] == 'preliminary']

In [5]:
# Get aggregated number of votes for each political party
pei=results_val.groupby('Political affiliation')['Votes obtained'].sum()
pei=pei.to_frame()

In [6]:
#new_constituencies.head()

,Provinces,Old Constituencies,New Constituencies
0,Cardigan,Prince Edward Island,Prince Edward Island
1,Charlottetown,Prince Edward Island,Prince Edward Island
2,Egmont,Prince Edward Island,Prince Edward Island
3,Malpeque,Prince Edward Island,Prince Edward Island
4,Avalon,Newfoundland and Labrador,Newfoundland and Labrador


In [7]:
#results_val.head()

,Electoral district number,Electoral district name,Type of results,Surname,Middle name(s),Given name,Political affiliation,Appartenance politique,Votes obtained,% Votes obtained,Rejected ballots,Total number of ballots cast
4,10001,Avalon,validated,Chapman,NaN,Matthew,Conservative,Conservateur,12855.0,31.1,397.0,41731.0
5,10001,Avalon,validated,Malone,NaN,Greg,Green Party,Parti Vert,2215.0,5.4,397.0,41731.0
6,10001,Avalon,validated,McDonald,NaN,Kenneth,Liberal,Libéral,19122.0,46.3,397.0,41731.0
7,10001,Avalon,validated,Movelle,NaN,Lea Mary,NDP-New Democratic Party,NPD-Nouveau Parti démocratique,7142.0,17.3,397.0,41731.0
12,10002,Bonavista--Burin--Trinity,validated,Cooper,NaN,Matthew,NDP-New Democratic Party,NPD-Nouveau Parti démocratique,3855.0,12.0,609.0,32788.0


In [8]:
#Merge the validated election results with the new_constituencies table. This essentially assigns newly created constituencies discussed in our research to vote based on the Provice from where the vote came.
df_combo = pd.merge(results_val,new_constituencies, left_on="Electoral district name",right_on ="Provinces")

In [9]:
#df_combo.tail()

,Electoral district number,Electoral district name,Type of results,Surname,Middle name(s),Given name,Political affiliation,Appartenance politique,Votes obtained,% Votes obtained,Rejected ballots,Total number of ballots cast,Provinces,Old Constituencies,New Constituencies
958,61001,Northwest Territories,validated,Quinlan,NaN,Luke,People's Party,Parti populaire,296.0,1.8,125.0,16416.0,Northwest Territories,Territories,Northwest Territories
959,62001,Nunavut,validated,Aglukkaq,NaN,Leona,Conservative,Conservateur,2469.0,26.1,88.0,9542.0,Nunavut,Territories,Nunavut
960,62001,Nunavut,validated,Pizzo Lyall,NaN,Megan,Liberal,Libéral,2918.0,30.9,88.0,9542.0,Nunavut,Territories,Nunavut
961,62001,Nunavut,validated,Qaqqaq,NaN,Mumilaaq,NDP-New Democratic Party,NPD-Nouveau Parti démocratique,3861.0,40.8,88.0,9542.0,Nunavut,Territories,Nunavut
962,62001,Nunavut,validated,Roy,NaN,Douglas,Green Party,Parti Vert,206.0,2.2,88.0,9542.0,Nunavut,Territories,Nunavut


In [13]:
#Drop columns that will not be used
df_combo=df_combo.drop(['Provinces','Appartenance politique','% Votes obtained'], axis = 1)
df_combo1 = df_combo.drop(['Type of results','Surname','Given name','Rejected ballots','Total number of ballots cast'], axis = 1)

#df_combo1.head()
df_combo1 = df_combo1.drop(["Middle name(s) "], axis = 1)

In [15]:
#Aggregate the votes obtained per party by each of the new constituencies that are discussed in the research
votesgrouped = df_combo1.groupby(['Political affiliation', 'New Constituencies'])["Votes obtained"].apply(lambda x : x.astype(int).sum())

In [17]:
votesgrouped.to_csv('Votes.csv')
votesgrouped = pd.read_csv('Vote.csv')

In [18]:
#votesgrouped.head()

,Political affiliation,New Constituencies,Votes obtained
0,Animal Protection Party,Calgary,247
1,Animal Protection Party,Eastern Townships,248
2,Animal Protection Party,Fredericton and North New Brunswick,286
3,Animal Protection Party,Greater Sudbury and Nipissing,282
4,Animal Protection Party,Halifax and West Nova (new constituencies),222


In [43]:
#Only select the parties that are classified as popular parties
array = ['Liberal','Conservative','Bloc Québécois','Green Party','NDP-New Democratic Party']
votesgrp1=votesgrouped.loc[votesgrouped['Political affiliation'].isin(array)]
#votesgrp1=votesgrouped

In [44]:
pei_new=votesgrp1.groupby('Political affiliation')['Votes obtained'].sum()
pei_new=pei_new.to_frame()

In [45]:
#pei_new.tail()

,Votes obtained
Political affiliation,
Bloc Québécois,672608
Conservative,2533546
Green Party,496318
Liberal,2791960
NDP-New Democratic Party,1279011


In [46]:
#rename the original Votes Obtained column to Votes_Seat1
pei_new.rename(columns={'Votes obtained': 'Votes_Seat1'}, inplace=True)

In this section we implement the D'Hondt formula. 
The requirement for this implementation is that there will be atleast one seat assignment. This implementation then allows the user to see the seat distributions between various parties based on any number of seat assignments.

In [ ]:
#Enter the number of seat assignments
ip = input("Enter number of seats : ") 

In [57]:
first_max_new = pei_new['Votes_Seat1'].max()
seat_new = 0
pei_new['Seats_Seat1'] = seat_new
pei_new.reset_index(inplace=True)
pei_new.loc[pei_new['Votes_Seat1'] == first_max_new, 'Seats_Seat1'] = 1

seats = int(ip)
for i in range (2,(seats+1)):
    col_name_votes = 'Votes_Seat'+str(i)
    col_name_votes_prev = 'Votes_Seat'+str(i-1)
    col_name_seats = 'Seats_Seat'+str(i)
    col_name_seats_prev = 'Seats_Seat'+str(i-1)
    pei_new[col_name_votes]=pei_new['Votes_Seat1']/(1+pei_new[col_name_seats_prev])
    max_val = pei_new[col_name_votes].max()
    pei_new[col_name_seats] = pei_new[col_name_seats_prev]
    pei_new.loc[pei_new[col_name_votes] == max_val, col_name_seats] = pei_new[col_name_seats_prev] + 1

In [58]:
pei_new.to_csv('New Constituencies Summary.csv',encoding='utf-8-sig')